In [1]:
Experiments_file='/home/kate/Research/YearBuilt/Experiments/DwellingExperiments.xlsx'
AllExperiments_tab='Experiments'
Experiment_name='FMultiPol'
#Experiment configuration: differenet datasets to try
#each line in the file contains the model name and set of features to built a dataset for SageMaker
Experiment_tab='%s InputData'%Experiment_name


Trial_name='%s-TrainingModels'%Experiment_name 

#BaseModel name to compare other models
BaseModel='BaseModel'

#model in
bucket='kdproperty'

#model parameters
hyperparameters = {
'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'silent': True,
        'booster': 'gbtree',
        'seed': 42,
        'scale_pos_weight':0.3,
        'colsample_bylevel': 0.8,
        'colsample_bytree': 0.8,
        'eta': 0.03,
        'max_depth': 4,
        'num_round':'5000'
}
#instance type to be created for training jobs.    
instance_type='ml.c5.xlarge'
#Script to be load in training instance
training_job_entry_point='ModelTraining.py'
#data in training and testing file type
content_type='text/csv'

In [2]:
#packages used in the notebook
import time
import sys
import sagemaker
from sagemaker.session import s3_input
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.xgboost.model import XGBoostModel
import pandas as pd
import numpy as np
from sagemaker.analytics import ExperimentAnalytics

In [3]:
#should be run as a first step
#role arn is used when run from a local machine
role = 'arn:aws:iam::757107622481:role/service-role/AmazonSageMaker-ExecutionRole-20200819T131882'
region = 'us-west-2'
sagemaker_session = sagemaker.session.Session(default_bucket=bucket)

In [4]:
sys.path.append('/home/kate/Research/YearBuilt/Notebooks/Property')
import ExperimentsUtils as eu

In [5]:
models = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=Experiment_tab)
models

,Model,Trial Component,Training_data,Validation_data
0,Landlord_9,FMultiPol-PreparingTrainValidData-Landlord,s3://kdproperty/Data/training_data/Landlord/fo...,s3://kdproperty/Data/validation_data/Landlord/...
1,Landlord_8,FMultiPol-PreparingTrainValidData-Landlord,s3://kdproperty/Data/training_data/Landlord/fo...,s3://kdproperty/Data/validation_data/Landlord/...
2,Landlord_7,FMultiPol-PreparingTrainValidData-Landlord,s3://kdproperty/Data/training_data/Landlord/fo...,s3://kdproperty/Data/validation_data/Landlord/...
3,Landlord_6,FMultiPol-PreparingTrainValidData-Landlord,s3://kdproperty/Data/training_data/Landlord/fo...,s3://kdproperty/Data/validation_data/Landlord/...
4,Landlord_5,FMultiPol-PreparingTrainValidData-Landlord,s3://kdproperty/Data/training_data/Landlord/fo...,s3://kdproperty/Data/validation_data/Landlord/...
5,Landlord_4,FMultiPol-PreparingTrainValidData-Landlord,s3://kdproperty/Data/training_data/Landlord/fo...,s3://kdproperty/Data/validation_data/Landlord/...
6,Landlord_3,FMultiPol-PreparingTrainValidData-Landlord,s3://kdproperty/Data/training_data/Landlord/fo...,s3://kdproperty/Data/validation_data/Landlord/...
7,Landlord_2,FMultiPol-PreparingTrainValidData-Landlord,s3://kdproperty/Data/training_data/Landlord/fo...,s3://kdproperty/Data/validation_data/Landlord/...
8,Landlord_1,FMultiPol-PreparingTrainValidData-Landlord,s3://kdproperty/Data/training_data/Landlord/fo...,s3://kdproperty/Data/validation_data/Landlord/...
9,Landlord_0,FMultiPol-PreparingTrainValidData-Landlord,s3://kdproperty/Data/training_data/Landlord/fo...,s3://kdproperty/Data/validation_data/Landlord/...


In [6]:
experiments = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=AllExperiments_tab)

In [7]:
#delete experiment and trials but not output files associated with jobs in experiments
#eu.cleanup_experiment(Experiment_name)
eu.cleanup_trial(Experiment_name, Trial_name)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


ActiveOtherPolicies-0-2020-12-30-22-12-17-aws-training-job


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


ActiveOtherPolicies-1-2020-12-30-22-12-15-aws-training-job


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


...

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Landlord-8-2020-12-30-22-11-29-aws-training-job


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Landlord-9-2020-12-30-22-11-28-aws-training-job


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [8]:
eu.create_experiment(Experiment_name)
eu.create_trial(Experiment_name,Trial_name)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [9]:
%%writefile ModelTraining.py

#  Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License").
#  You may not use this file except in compliance with the License.
#  A copy of the License is located at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  or in the "license" file accompanying this file. This file is distributed
#  on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
#  express or implied. See the License for the specific language governing
#  permissions and limitations under the License.
from __future__ import print_function

import argparse
import json
import logging
import os
import pandas as pd
import pickle as pkl

from sagemaker_containers import entry_point
from sagemaker_xgboost_container.data_utils import get_dmatrix
from sagemaker_xgboost_container import distributed

import xgboost as xgb

import pandas as pd
import numpy as np


def _xgb_train(params, dtrain, evals, num_boost_round, model_dir, is_master):
    """Run xgb train on arguments given with rabit initialized.

    This is our rabit execution function.

    :param args_dict: Argument dictionary used to run xgb.train().
    :param is_master: True if current node is master host in distributed training,
                        or is running single node training job.
                        Note that rabit_run will include this argument.
    """
    booster = xgb.train(params=params,
                        dtrain=dtrain,
                        evals=evals,
                        maximize=True,
                        num_boost_round=num_boost_round,
                        early_stopping_rounds=100,
                        verbose_eval=100)

    if is_master:
        model_location = model_dir + '/xgboost-model'
        pkl.dump(booster, open(model_location, 'wb'))
        logging.info("Stored trained model at {}".format(model_location))


if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Hyperparameters are described here.
    parser.add_argument('--max_depth', type=int,)
    parser.add_argument('--eta', type=float)
    parser.add_argument('--objective', type=str)
    parser.add_argument('--num_round', type=int)
    
    parser.add_argument('--eval_metric', type=str)
    parser.add_argument('--seed', type=float)
    parser.add_argument('--scale_pos_weight', type=float)
    parser.add_argument('--colsample_bylevel', type=float)
    parser.add_argument('--colsample_bytree', type=float)
        
            
            

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--output_data_dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))
    parser.add_argument('--sm_hosts', type=str, default=os.environ.get('SM_HOSTS'))
    parser.add_argument('--sm_current_host', type=str, default=os.environ.get('SM_CURRENT_HOST'))

    args, _ = parser.parse_known_args()

    # Get SageMaker host information from runtime environment variables
    sm_hosts = json.loads(args.sm_hosts)
    sm_current_host = args.sm_current_host

    dtrain = get_dmatrix(args.train, 'csv')
    dval = get_dmatrix(args.validation, 'csv')
    watchlist = [(dtrain, 'train'), (dval, 'validation')] if dval is not None else [(dtrain, 'train')]

    
    
    train_hp = {
        'max_depth': args.max_depth,
        'eta': args.eta,
        'objective': args.objective,
        'silent': True,
        'booster': 'gbtree',
        'seed': 42,
        'eval_metric':args.eval_metric,
        'scale_pos_weight':args.scale_pos_weight,
        'colsample_bylevel': args.colsample_bylevel,
        'colsample_bytree': args.colsample_bytree
        }

    xgb_train_args = dict(
        params=train_hp,
        dtrain=dtrain,
        evals=watchlist,
        num_boost_round=args.num_round,
        model_dir=args.model_dir)

    if len(sm_hosts) > 1:
        # Wait until all hosts are able to find each other
        entry_point._wait_hostname_resolution()

        # Execute training function after initializing rabit.
        distributed.rabit_run(
            exec_fun=_xgb_train,
            args=xgb_train_args,
            include_in_training=(dtrain is not None),
            hosts=sm_hosts,
            current_host=sm_current_host,
            update_rabit_args=True
        )
    else:
        # If single node training, call training method directly.
        if dtrain:
            xgb_train_args['is_master'] = True
            _xgb_train(**xgb_train_args)
        else:
            raise ValueError("Training channel must have data to train model.")


def model_fn(model_dir):
    """Deserialize and return fitted model.

    Note that this should have the same name as the serialized model in the _xgb_train method
    """
    model_file = 'xgboost-model'
    booster = pkl.load(open(os.path.join(model_dir, model_file), 'rb'))
    return booster

Overwriting ModelTraining.py


In [12]:

processors=list()
i = 0
for index, row in models.iterrows():
    model=row['Model']
    print (index, ': Training model %s'%model)
    print(row['Training_data'])
    print(row['Validation_data'])
    train_input = s3_input(row['Training_data'], content_type=content_type)
    validation_input = s3_input(row['Validation_data'], content_type=content_type)
                               
    xgb_script_mode_estimator = XGBoost(
        entry_point=training_job_entry_point,
        hyperparameters=hyperparameters,
        role=role, 
        train_instance_count=1,
        train_instance_type=instance_type,
        framework_version="1.0-1",
        output_path='s3://%s'%(bucket)
        )
    #Training
    job_name=model.replace('_','-') +'-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
    xgb_script_mode_estimator.fit({'train': train_input, 'validation': validation_input}, job_name=job_name, wait=False,
    experiment_config = {
        'ExperimentName': Experiment_name ,
        'TrialName' : Trial_name,
        'TrialComponentDisplayName' : '%s-%s'%(Trial_name,model.replace('_','-')),})
    # to prevent throttling
    time.sleep(1)
    processors.append(xgb_script_mode_estimator)
    i=i+1
    #LIMIT ONLY 30 running jobs for ml.c5.xlarge! Wait till the first portion is done ~40 min
    if i==30:
        time.sleep(40*60)
        i=0

In [ ]:
eu.wait_training_jobs(processors=processors,check_every_sec=10,print_every_n_output=10,wait_min=60)

In [14]:
trial_component_analytics = ExperimentAnalytics(
    experiment_name=Experiment_name
)
trial_comp_ds = trial_component_analytics.dataframe()
trial_ds=trial_comp_ds[trial_comp_ds['DisplayName'].str.contains(Trial_name)].copy()
trial_ds['Model']=trial_ds['DisplayName'].str.replace(Trial_name+'-','')
trial_ds['Model']=trial_ds['Model'].str.replace('-','_')

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [15]:
#hyperparameters used in models
ModelsParams = trial_ds[['Model','DisplayName',
                                                                      'objective', 
                                                                      'eval_metric', 
                                                                      'silent',  
                                                                      'booster', 
                                                                      'seed',
                                                                      'scale_pos_weight', 
                                                                      'colsample_bylevel',
                                                                      'colsample_bytree',
                                                                      'eta',
                                                                      'max_depth', 
                                                                      'num_round']]
ModelsParams.columns=['Model','Trial Component',
                                                                      'objective', 
                                                                      'eval_metric', 
                                                                      'silent',  
                                                                      'booster', 
                                                                      'seed',
                                                                      'scale_pos_weight', 
                                                                      'colsample_bylevel',
                                                                      'colsample_bytree',
                                                                      'eta',
                                                                      'max_depth', 
                                                                      'num_round']
ModelsParams.sort_values('Model', ascending=True)

,Model,Trial Component,objective,eval_metric,silent,booster,seed,scale_pos_weight,colsample_bylevel,colsample_bytree,eta,max_depth,num_round
8,ActiveOtherPolicies_0,FMultiPol-TrainingModels-ActiveOtherPolicies-0,"""binary:logistic""","""auc""",true,"""gbtree""",42.0,0.3,0.8,0.8,0.03,4.0,"""5000"""
26,ActiveOtherPolicies_1,FMultiPol-TrainingModels-ActiveOtherPolicies-1,"""binary:logistic""","""auc""",true,"""gbtree""",42.0,0.3,0.8,0.8,0.03,4.0,"""5000"""
17,ActiveOtherPolicies_2,FMultiPol-TrainingModels-ActiveOtherPolicies-2,"""binary:logistic""","""auc""",true,"""gbtree""",42.0,0.3,0.8,0.8,0.03,4.0,"""5000"""
10,ActiveOtherPolicies_3,FMultiPol-TrainingModels-ActiveOtherPolicies-3,"""binary:logistic""","""auc""",true,"""gbtree""",42.0,0.3,0.8,0.8,0.03,4.0,"""5000"""
18,ActiveOtherPolicies_4,FMultiPol-TrainingModels-ActiveOtherPolicies-4,"""binary:logistic""","""auc""",true,"""gbtree""",42.0,0.3,0.8,0.8,0.03,4.0,"""5000"""
5,ActiveOtherPolicies_5,FMultiPol-TrainingModels-ActiveOtherPolicies-5,"""binary:logistic""","""auc""",true,"""gbtree""",42.0,0.3,0.8,0.8,0.03,4.0,"""5000"""
11,ActiveOtherPolicies_6,FMultiPol-TrainingModels-ActiveOtherPolicies-6,"""binary:logistic""","""auc""",true,"""gbtree""",42.0,0.3,0.8,0.8,0.03,4.0,"""5000"""
7,ActiveOtherPolicies_7,FMultiPol-TrainingModels-ActiveOtherPolicies-7,"""binary:logistic""","""auc""",true,"""gbtree""",42.0,0.3,0.8,0.8,0.03,4.0,"""5000"""
23,ActiveOtherPolicies_8,FMultiPol-TrainingModels-ActiveOtherPolicies-8,"""binary:logistic""","""auc""",true,"""gbtree""",42.0,0.3,0.8,0.8,0.03,4.0,"""5000"""
1,ActiveOtherPolicies_9,FMultiPol-TrainingModels-ActiveOtherPolicies-9,"""binary:logistic""","""auc""",true,"""gbtree""",42.0,0.3,0.8,0.8,0.03,4.0,"""5000"""


In [16]:
#Saving into the Experiment log file models hyperparameters used
eu.SaveToExperimentLog(Experiments_file, '%s ModelParams'%Experiment_name, ModelsParams)

In [ ]:
#Reading models results and formatting data for analysis

In [17]:
#Validation results
ModelsResults = trial_ds[['Model','validation:auc - Last']].copy()
ModelsResults['#']=ModelsResults['Model'].apply(lambda x: x[::-1][:1])
ModelsResults['Model']=ModelsResults['Model'].apply(lambda x: x[:len(x)-2])
ModelsResults.columns=['Model','validation:auc','#']
ModelsResults=ModelsResults[['Model','#','validation:auc']]
ModelsResults.sort_values(['Model','#'], ascending=[False,True])
ModelsResults = pd.pivot_table(ModelsResults, index=['Model'], columns=['#'])
ModelsResults.reset_index( drop=False, inplace=True )
ModelsResults.columns = ModelsResults.columns.droplevel(0)
ModelsResults.columns =['Model','0','1','2','3','4','5','6','7','8','9']
ModelsResults['Mean']=ModelsResults[['0','1','2','3','4','5','6','7','8','9']].mean(axis=1)
ModelsResults['t-statistic']=0.000
ModelsResults['pvalue']=0.000
ModelsResults['Comment']='Base Model'
ModelsResults

,Model,0,1,2,3,4,5,6,7,8,9,Mean,t-statistic,pvalue,Comment
0,ActiveOtherPolicies,0.65999,0.66084,0.50000,0.63392,0.65693,0.62777,0.67448,0.66006,0.61836,0.68041,0.637276,0.0,0.0,Base Model
1,BaseModel,0.64649,0.64168,0.65413,0.63699,0.65487,0.60383,0.66633,0.65286,0.64065,0.50000,0.629783,0.0,0.0,Base Model
2,Landlord,0.65205,0.65367,0.66095,0.63282,0.65464,0.61681,0.65559,0.62105,0.64289,0.66414,0.645461,0.0,0.0,Base Model


In [18]:
import scipy.stats as stats

In [26]:
#BaseModel='BaseModel'
BaseModelResults=ModelsResults[ModelsResults['Model']==BaseModel][['0','1','2','3','4','5','6','7','8','9']].values[0].tolist()

In [27]:
for index, model in ModelsResults.iterrows():
    if model['Model']!=BaseModel:
        AnalyzedModelResults=model[['0','1','2','3','4','5','6','7','8','9']].values.tolist()
        t=stats.ttest_ind(BaseModelResults,AnalyzedModelResults)
        ModelsResults.at[index,'t-statistic']= t.statistic
        ModelsResults.at[index,'pvalue'] = t.pvalue 
        if t.pvalue>=0.05:
            ModelsResults.at[index,'Comment'] = 'No difference with %s with 0.05 confidence level'%BaseModel
        else:
            ModelsResults.at[index,'Comment'] = 'There is a difference with %s with 0.05 confidence level'%BaseModel
ModelsResults

,Model,0,1,2,3,4,5,6,7,8,9,Mean,t-statistic,pvalue,Comment
0,ActiveOtherPolicies,0.65999,0.66084,0.50000,0.63392,0.65693,0.62777,0.67448,0.66006,0.61836,0.68041,0.637276,-0.332435,0.743401,No difference with BaseModel with 0.05 confide...
1,BaseModel,0.64649,0.64168,0.65413,0.63699,0.65487,0.60383,0.66633,0.65286,0.64065,0.50000,0.629783,0.966790,0.346460,No difference with Landlord with 0.05 confiden...
2,Landlord,0.65205,0.65367,0.66095,0.63282,0.65464,0.61681,0.65559,0.62105,0.64289,0.66414,0.645461,-0.966790,0.346460,No difference with BaseModel with 0.05 confide...


In [28]:
#Saving into the Experiment log file models results
eu.SaveToExperimentLog(Experiments_file, '%s ModelComparing'%Experiment_name, ModelsResults)

In [29]:
#Models
ModelsFiles = trial_ds[['Model','DisplayName','SageMaker.ModelArtifact - Value']]
ModelsFiles.columns=['Model','Trial Component','ModelData']
ModelsFiles

,Model,Trial Component,ModelData
0,Landlord_2,FMultiPol-TrainingModels-Landlord-2,s3://kdproperty/Landlord-2-2020-12-31-19-32-33...
1,ActiveOtherPolicies_9,FMultiPol-TrainingModels-ActiveOtherPolicies-9,s3://kdproperty/ActiveOtherPolicies-9-2020-12-...
2,Landlord_4,FMultiPol-TrainingModels-Landlord-4,s3://kdproperty/Landlord-4-2020-12-31-19-32-29...
3,BaseModel_5,FMultiPol-TrainingModels-BaseModel-5,s3://kdproperty/BaseModel-5-2020-12-31-19-32-4...
4,Landlord_7,FMultiPol-TrainingModels-Landlord-7,s3://kdproperty/Landlord-7-2020-12-31-19-32-23...
5,ActiveOtherPolicies_5,FMultiPol-TrainingModels-ActiveOtherPolicies-5,s3://kdproperty/ActiveOtherPolicies-5-2020-12-...
6,BaseModel_6,FMultiPol-TrainingModels-BaseModel-6,s3://kdproperty/BaseModel-6-2020-12-31-19-32-4...
7,ActiveOtherPolicies_7,FMultiPol-TrainingModels-ActiveOtherPolicies-7,s3://kdproperty/ActiveOtherPolicies-7-2020-12-...
8,ActiveOtherPolicies_0,FMultiPol-TrainingModels-ActiveOtherPolicies-0,s3://kdproperty/ActiveOtherPolicies-0-2020-12-...
9,BaseModel_0,FMultiPol-TrainingModels-BaseModel-0,s3://kdproperty/BaseModel-0-2020-12-31-19-32-5...


In [30]:
#Saving into the Experiment log file models files
eu.SaveToExperimentLog(Experiments_file, '%s ModelFiles'%Experiment_name, ModelsFiles)